In [2]:
from huggingface_hub import whoami

try:
    user_info = whoami()
    print(f"로그인 상태입니다. 사용자: {user_info['name']}")
except Exception as e:
    print("로그인되지 않았거나 토큰이 유효하지 않습니다.")
    print(e)

/home/chae/faiss_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


로그인 상태입니다. 사용자: chaeeee


In [3]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "false"

In [4]:
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
import faiss  # FAISS 라이브러리 필요

# 임베딩 초기화
embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")


You try to use a model that was created with version 3.3.1, however, your version is 3.2.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [3]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import fitz  # PyMuPDF
import pdfplumber
from langchain_core.documents import Document
from langchain.docstore.in_memory import InMemoryDocstore


def extract_text_paragraphs(pdf_path):
    """
    PDF 문서에서 문단별로 텍스트 추출 (표는 제외)
    """
    doc_list = []
    pdf_doc = fitz.open(pdf_path)  # PyMuPDF로 PDF 열기

    with pdfplumber.open(pdf_path) as pdf:
        for page_num in range(len(pdf_doc)):
            page = pdf_doc[page_num]

            # 텍스트 추출 (문단 단위로 구분)
            text = page.get_text("text")
            paragraphs = text.split("\n\n")  # 빈 줄로 문단 구분

            # 표 추출
            tables = pdf.pages[page_num].extract_tables()
            table_texts = []
            if tables:
                for table in tables:
                    df = pd.DataFrame(table[1:], columns=table[0])  # 첫 행을 컬럼으로 설정
                    table_texts.append(df.to_string())
            else:
                table_texts = []  # 표가 없는 경우 빈 리스트

            # 문서 객체 생성 (문단별로 추가)
            for para in paragraphs:
                doc = Document(
                    page_content=para + "\n\n" + "\n\n".join(table_texts),  # 문단 + 표 포함
                    metadata={"page": page_num + 1}  # 페이지 정보 포함
                )
                doc_list.append(doc)

    return doc_list

In [4]:
def create_faiss_index(documents):
    """
    문서에서 텍스트를 추출하여 벡터화하고, 이를 FAISS 인덱스에 추가합니다.
    """
    texts = [doc.page_content for doc in documents]  # 문서에서 텍스트 추출
    embeddings = embedding_model.encode(texts)  # SentenceTransformer로 텍스트를 벡터화

    # FAISS 인덱스 생성
    embedding_dim = embeddings.shape[1]  # 벡터 차원
    faiss_index = faiss.IndexFlatL2(embedding_dim)  # L2 거리 기반 인덱스 생성
    faiss_index.add(embeddings)  # 벡터 추가

    return faiss_index, embeddings

In [5]:
def create_docstore(documents):
    """
    문서 객체를 InMemoryDocstore에 저장합니다.
    """
    docstore = InMemoryDocstore({i: doc for i, doc in enumerate(documents)})
    return docstore

In [12]:
pdf_path = './PDF/dataset/artworks_10.pdf'
documents = extract_text_paragraphs(pdf_path)
faiss_index, embeddings = create_faiss_index(documents)

In [13]:
docstore = create_docstore(documents)

In [42]:
documents

[Document(metadata={'page': 1}, page_content='10\n10\n\n\n        10    10\n0  None     None'),
 Document(metadata={'page': 2}, page_content='10\n\n\n'),
 Document(metadata={'page': 3}, page_content='논문\n과학적 분석을 통한 유화 작품의 하부 도상 제작 시기 검증연구 \n: 오지호 <풍경>(1927), <녹음>(1975)을 중심으로 / 5\n정참희, 박혜선, 이나라, 김정흠, 임성진\n오브제와 결합한 뉴미디어 작품의 과학적 분석 및 보존처리\n: 백남준 作 <시바> / 31\n정다연, 이의천, 명나희, 이상훈, 권인철\n목제 조각의 수종 분석 및 보존처리 \n: 이종각 作 <확산공간> / 51\n이의천, 김영목, 조상윤, 신정아, 김이연, 권희홍\n미술품의 보존을 위한 전시실 보존환경 연구 / 69\n송송이, 주창하, 범대건\nEnzo Mari의 “자연 시리즈 serie della natura”의 \nAquazolⓇ을 사용한 색맞춤 보존 처리 연구 / 95\n박자현\n자료\n유토로 제작된 개념미술 작품의 보존처리 / 107\n조인애, 문희경, 범대건\n근대문화유산 보존처리의 현황과 과제 / 121\n박형호\n복원 모사를 통한 회화작품의 보존 / 135\n정두희\n목차\n\n\n'),
 Document(metadata={'page': 4}, page_content="과학적 분석을 통한 유화 작품의 \n하부 도상 제작 시기 검증연구 : 오지호 <풍경>(1927), <녹음>(1975)을 중심으로\n5\n과학적 분석을 통한 유화 작품의 \n하부 도상 제작 시기 검증연구 \n: 오지호 <풍경>(1927), <녹음>(1975)을 중심으로\n정참희, 박혜선, 이나라, 김정흠*, 임성진\n국립현대미술관 작품보존미술은행관리과\n Verification study on the production period of unde

In [16]:
def embed_query(text):
    return embedding_model.encode([text])[0]  # 단일 쿼리 텍스트를 임베딩

index_to_docstore_id = {i: i for i in range(len(documents))}

faiss_db = FAISS(
    embedding_function=embed_query,
    index=faiss_index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

# 7. FAISS 데이터베이스 저장
faiss_db.save_local("./faiss_artworks_0304")
print("FAISS 데이터베이스가 성공적으로 저장되었습니다!")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


FAISS 데이터베이스가 성공적으로 저장되었습니다!


In [17]:
# 9. 검색 테스트
query = "실내 공기질 측정항목에는 무엇이 있어?"
results = faiss_db.similarity_search(query, k=5)

# 10. 검색 결과 출력
for result in results:
    print("문서 텍스트:", result.page_content)
    print("문서 메타데이터:", result.metadata)
    print('=------------------------------------------')


문서 텍스트: 미술작품의 보존 10
미술품의 보존을 위한 전시실 보존환경 연구
77
76
Table 5. The causes of indoor pollution and their effects on human health 
Pollutant
Cause
Effect on Human Health
PM10
Combustion of fuels, automobiles
Allergic conjunctivitis, rhinitis
CO₂
Combustion of fossil fuels, respiratory system 
of organisms
Drowsiness, headaches, dizziness
HCHO
Preservatives, synthetic resin products, 
building materials
Dizziness, vomiting, sick building syndrome
TAB
Food waste, organic matter, 
humid environment
Allergies, respiratory diseases
CO
Exhaust fumes, heating devices
Headaches, dizziness, vomiting
NO₂
Automobile engines, boilers
Cardiovascular diseases, respiratory 
diseases
Rn
Soil, groundwater, building materials
Lung cancer
VOCs
Industrial solvent usage, organic gases
Drowsiness, headaches, dizziness
Mold
Moist environment, food, building finishing 
materials
Eczema, skin disorders, inflammation
실내에서 발생하는 오염물질은 석고, 시멘트 분자, 방음재, 바닥재, 목재 패널 등의 
건축 자재와 페인트 및 청소 제품에서 폼알데하이드와 아세트알데하이드를 발생시키며 
내부 설비용 

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4bit 양자화 활성화
    bnb_4bit_compute_dtype=torch.float16,  # 계산 타입 설정 (float16이 일반적)
    bnb_4bit_use_double_quant=True,  # 더블 양자화 사용 (메모리 절약)
    bnb_4bit_quant_type="nf4",  # NormalFloat4 (NF4) 사용
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit 양자화 활성화
    bnb_4bit_compute_dtype="float16",  # 계산 정밀도 설정
    bnb_4bit_quant_type="nf4",  # NF4 양자화 방식 사용 (효율적)
    bnb_4bit_use_double_quant=True,  # 이중 양자화 사용
)

In [8]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# 모델과 토크나이저 로드 (CUDA 사용)
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,  # ✅ 올바른 양자화 설정 적용
    device_map="auto",  # ✅ 자동 GPU 배치
    trust_remote_code=True,
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:27<00:00,  6.97s/it]


In [9]:
from transformers import pipeline

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
llm = HuggingFacePipeline(pipeline=pipe)

/tmp/ipykernel_1010718/1315975019.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [10]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# 🔹 EXAONE 모델 로드
exaone_model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"
exaone_tokenizer = AutoTokenizer.from_pretrained(exaone_model_id)
exaone_model = AutoModelForCausalLM.from_pretrained(
    exaone_model_id,
    quantization_config=quantization_config,
    device_map="cuda",  # CUDA에서 자동 배치
    trust_remote_code=True
)


Loading checkpoint shards: 100%|██████████| 7/7 [00:21<00:00,  3.01s/it]


In [11]:
from transformers import pipeline

# 파이프라인 생성
exaone_pipe = pipeline(
    "text-generation",
    model=exaone_model,
    tokenizer=exaone_tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
exaone_llm = HuggingFacePipeline(pipeline=exaone_pipe)

In [12]:
from langchain.prompts import ChatPromptTemplate

deepseek_template = """
<|system|>
You are a friendly chatbot specializing in artworks and general conversations.
Your primary role is to answer questions **accurately based on the provided document (context)**. 
If the requested information is not found in the document, respond with:
"문서에 해당 정보가 없습니다." 

However, if the question is a general conversation or does not relate to the document, you should respond naturally as a conversational chatbot. 
You can talk about art history, artists, exhibitions, and general topics such as daily life, technology, and culture. 
Maintain a friendly and engaging tone, ensuring all responses are written in Korean.
Use **beautiful Markdown formatting** (headings, bullet points, **bold** or *italic* text) to enhance readability.
You must include the artwork number in your response only when it is a question about an artwork.

<|context|>
{context}

<|user|>
Question: {question}

<|assistant|>
"""




exaone_template = '''
<|system|>
You are an AI assistant tasked with refining and polishing the provided logical reasoning into a final answer in Korean.  
Your role is to produce a clear, concise, and well-structured response that maintains the original meaning and key details.  
Ensure that your final answer is written in Korean and uses **beautiful Markdown formatting** (e.g., headings, bullet points, **bold** or *italic* text) to enhance readability.  
Focus solely on refining the content without adding any new information.
You must include the artwork number in your response only when it is a question about an artwork.

<|reasoning|>
{reasoning}

<|user|>
Based on the above reasoning, please generate a refined and final answer in Korean.

<|assistant|>
'''



# DeepSeek 템플릿 생성
deepseek_prompt = ChatPromptTemplate.from_template(deepseek_template)

# EXAONE 템플릿 생성
exaone_prompt = ChatPromptTemplate.from_template(exaone_template)



In [47]:
retriever = faiss_db.as_retriever(
    search_kwargs={
        "k": 5,                # 검색 결과 개수
        "fetch_k": 15,         # 더 많은 결과 가져오기
        "mmr": True,           # MMR 활성화
        "mmr_beta": 0.3      # 다양성과 관련성 간 균형
    }
)


In [48]:
import re

class MarkdownOutputParser:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        """Extracts the assistant's response from after the </think> tag and formats it in Markdown."""
        if not llm_output or llm_output.strip() == "":
            return "❌ 모델에서 응답을 생성하지 못했습니다."

        # "</think>" 이후 텍스트 추출
        match = re.search(r"</think>\s*(.*)", llm_output, re.DOTALL)
        extracted_text = match.group(1).strip() if match else llm_output.strip()

        # Markdown 형식 적용
        formatted_output = f"""
### **🔹 모델 응답 결과**

{extracted_text}
"""
        return formatted_output.strip()  # 양 끝 공백 제거


In [49]:
import re

class MarkdownOutputParser2:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        """Extracts the assistant's response from after the </think> tag and formats it in Markdown."""
        if not llm_output or llm_output.strip() == "":
            return "❌ 모델에서 응답을 생성하지 못했습니다."

        # "</think>" 이후 텍스트 추출
        match = re.search(r"<\|assistant\|>\s*(.*)", llm_output, re.DOTALL)
        extracted_text = match.group(1).strip() if match else llm_output.strip()

        # Markdown 형식 적용
        formatted_output = f"""
### **🔹 모델 응답 결과**

{extracted_text}
"""
        return formatted_output.strip()  # 양 끝 공백 제거


In [50]:
from langchain.schema.runnable import RunnableLambda

chain = (
    retriever
    | RunnableLambda(lambda docs: {  
        "context": "\n".join([doc.page_content for doc in docs]),  
        "question": query 
    })
    | deepseek_prompt
    | llm
    | MarkdownOutputParser()
    | (lambda x: {"reasoning": x})
    | exaone_prompt
    | exaone_llm
    | MarkdownOutputParser2()
)


In [63]:
query = "미술관에서 동양화의 권고조도 기준은 어떠한가?"

In [64]:
response = chain.invoke({"question": query})
print(response)

### **🔹 모델 응답 결과**

## 미술관 동양화 전시를 위한 조명 권장 사항

동양화 전시 시 **권고 조도는 100 lux 이하**로 유지해야 합니다. 이 수준의 조명은 다음과 같은 이점을 제공합니다:

* **작품의 변색 및 퇴색 방지:** 과도한 빛 노출로 인한 손상을 최소화합니다.
* **추천 조명 유형:**
    * **LED 조명:** 자외선 방출이 거의 없으며 장시간 사용에도 내구성이 뛰어납니다.

이러한 조명 조건 준수는 동양화의 보존과 아름다움 유지에 필수적입니다.


In [23]:
retrieved_docs = retriever.get_relevant_documents(query)
for i, doc in enumerate(retrieved_docs):
    print(f"Document {i+1}:")
    print(f"Content: {doc.page_content}")  # 문서의 실제 내용
    print(f"Metadata: {doc.metadata}")    # 메타데이터 (예: 출처, 페이지 등)
    print("-" * 50)


Document 1:
Content: 논문
과학적 분석을 통한 유화 작품의 하부 도상 제작 시기 검증연구 
: 오지호 <풍경>(1927), <녹음>(1975)을 중심으로 / 5
정참희, 박혜선, 이나라, 김정흠, 임성진
오브제와 결합한 뉴미디어 작품의 과학적 분석 및 보존처리
: 백남준 作 <시바> / 31
정다연, 이의천, 명나희, 이상훈, 권인철
목제 조각의 수종 분석 및 보존처리 
: 이종각 作 <확산공간> / 51
이의천, 김영목, 조상윤, 신정아, 김이연, 권희홍
미술품의 보존을 위한 전시실 보존환경 연구 / 69
송송이, 주창하, 범대건
Enzo Mari의 “자연 시리즈 serie della natura”의 
AquazolⓇ을 사용한 색맞춤 보존 처리 연구 / 95
박자현
자료
유토로 제작된 개념미술 작품의 보존처리 / 107
조인애, 문희경, 범대건
근대문화유산 보존처리의 현황과 과제 / 121
박형호
복원 모사를 통한 회화작품의 보존 / 135
정두희
목차



Metadata: {'page': 3}
--------------------------------------------------
Document 2:
Content: 미술작품의 보존 10
목제 조각의 수종 분석 및 보존처리 : 이종각 作 <확산공간>
57
56
(A) Dust and foreign substances
(B) Insect house between the crack (C) Spider webs and adhesive traces
Figure 5. Appearance before treatment of Jong Gak Lee’s <Diffusion space> sculpture(1). 
2번 조각은 크기 133(H)×100(W)×53(L)㎝, 중량 약 110㎏으로 표면에서 먼지 등 오염물과 
거미줄이 확인되었으며. 하단부에는 토양 오염물이 보였다(Figure 6A). 작품의 중심부에는 
할렬이 중-하단부로 길게 이어진 형태이며, 조각 후면에는 상-하단부 내부 공동(Figure

/tmp/ipykernel_1010718/3695306690.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


### DB 2개 연결

In [1]:
from langchain_community.vectorstores import FAISS


# 기존 DB 로드 
persist_directory = "./faiss_artworks_0304"

try:
    faiss_db = FAISS.load_local(
        folder_path=persist_directory,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  # 신뢰할 수 있는 소스에서만 사용
    )
    
    # embedding_function 수정
    faiss_db.embedding_function = lambda text: (
        embedding_model.encode(text) if isinstance(text, str) else embedding_model.encode(str(text))
    )
    
    print("FAISS 데이터베이스가 성공적으로 로드되었습니다!")
except Exception as e:
    print(f"FAISS 데이터베이스 로드 중 오류 발생: {e}")

FAISS 데이터베이스 로드 중 오류 발생: name 'embedding_model' is not defined


In [43]:
from langchain_community.vectorstores import FAISS


# 기존 DB 로드 
persist_directory = "./faiss_artworks_0114_docx"

try:
    faiss_db2 = FAISS.load_local(
        folder_path=persist_directory,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  # 신뢰할 수 있는 소스에서만 사용
    )
    
    # embedding_function 수정
    faiss_db2.embedding_function = lambda text: (
        embedding_model.encode(text) if isinstance(text, str) else embedding_model.encode(str(text))
    )
    
    print("FAISS 데이터베이스가 성공적으로 로드되었습니다!")
except Exception as e:
    print(f"FAISS 데이터베이스 로드 중 오류 발생: {e}")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


FAISS 데이터베이스가 성공적으로 로드되었습니다!


In [41]:
vectors_db, docs_db = faiss_db.index.reconstruct_n(0, faiss_db.index.ntotal), faiss_db.docstore._dict.values()


In [42]:
len(vectors_db)

81

In [44]:
vectors_db2, docs_db2 = faiss_db2.index.reconstruct_n(0, faiss_db2.index.ntotal), faiss_db2.docstore._dict.values()


# 첫 번째 FAISS DB에 두 번째 DB의 문서를 추가
faiss_db.add_documents(docs_db2, embeddings=vectors_db2)

['89ad7976-5e5e-4b76-9ccb-0d269c5de647',
 'ad4cfdc4-6614-44a1-9510-535869545e07',
 '2b077da9-75d3-4229-80e3-a7e44b8b4ef0',
 '6b165741-d44a-470e-9527-161abffd4286',
 'adfca107-b4b6-433d-aae3-8819206b8357',
 '98e995e1-1bb0-473a-9b01-fe34f36036d0',
 '865b3f5e-9128-4a60-9ddb-eca8d6ab4b7a',
 'd2e7c86b-3538-46c4-8263-388cd2a36923',
 '6a28a2fd-19c9-4fad-8d53-62e63a5eb770',
 'a7f7a5eb-4e89-454d-aa2f-38c43d16a106',
 '9033e6ee-8aa3-4844-9e42-beb44bfe43db',
 '433d00cb-51dc-42dc-b348-9f873c99cd51',
 '20b5258f-e0f7-432a-806a-3d2d1d3a71f5',
 '7831d4c8-092a-4957-aebd-c2e032645b55',
 '78fb9cf1-898d-4d12-865e-f1834905354f',
 'b02187d9-6de2-4da7-9fed-d263da15834a',
 'e4afe00d-da0f-411e-be57-dd2f028f5d5d',
 '688d5ba2-ab4c-44d3-bc54-f84cda3a9941',
 '4bdee590-e77d-4f33-8a52-f65045379ead',
 'eec5aa08-a0cc-4172-a36d-6cb170625ff1',
 '6aada8ce-7c37-4ceb-961e-7a7eea7b3107',
 '247b2ac8-c955-4fda-9f2b-24cd0b166afc',
 '8091ef44-3d4a-4475-8666-ecf8e0755a5f',
 'ba539f51-0ddb-4574-9c5b-9a39c132c816',
 '7c50730c-abd1-

In [45]:
faiss_db.save_local("faiss_combined")

In [46]:
# FAISS 인덱스에서 벡터 차원 확인
print(faiss_db.index.d)

1024


In [ ]:
query = "김기승의 낙양성동도이화에 대해 알려줘."

response = chain.invoke({"question": query})
print(response)

In [39]:
if faiss_db.index.d == vectors_db2.shape[1]:
    print("벡터 차원이 일치합니다.")
else:
    print("벡터 차원이 일치하지 않습니다.")


벡터 차원이 일치합니다.
